In [14]:
import pandas as pd
import numpy as np
import awoc
from sklearn import preprocessing

In [15]:
df = pd.read_csv("../LifeExpectancyData.csv")

In [16]:
columns_to_standardize = list(df.columns)[3:]

Here we do normalization.

We could maybe also do min-max normalization to get values between 0 and 1

In [17]:
for column in columns_to_standardize:
    mean = df[column].mean()
    std_dev = df[column].std()
    df[column] = (df[column] - mean) / std_dev

In [18]:
df

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,-0.443615,0.790103,0.268778,-1.133365,-0.335513,-0.635838,-0.110366,...,-3.267459,0.889322,-0.730453,-0.323390,-0.483449,0.343917,2.796324,2.756711,-0.704355,-0.563512
1,Afghanistan,2014,Developing,-0.979112,0.854468,0.285738,-1.133365,-0.334384,-0.755503,-0.168095,...,-1.047897,0.897327,-0.856945,-0.323390,-0.481456,-0.203661,2.864194,2.801068,-0.718580,-0.593284
2,Afghanistan,2013,Developing,-0.979112,0.830331,0.302697,-1.133365,-0.334537,-0.675726,-0.173502,...,-0.877162,0.877314,-0.772617,-0.323390,-0.480121,0.311058,2.909441,2.845424,-0.747029,-0.623055
3,Afghanistan,2012,Developing,-1.021112,0.862513,0.328137,-1.133365,-0.332040,-0.556061,0.032040,...,-0.663742,1.033419,-0.646125,-0.323390,-0.477443,-0.148436,2.954688,2.911960,-0.780219,-0.652827
4,Afghanistan,2011,Developing,-1.052611,0.886650,0.345097,-1.133365,-0.367800,-0.516173,0.051748,...,-0.621059,0.773244,-0.603961,-0.323390,-0.519939,-0.160210,3.022558,2.956317,-0.822893,-0.742141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,Zimbabwe,2004,Developing,-2.617102,4.491063,-0.028017,-0.059930,-0.371370,-0.516173,-0.208296,...,-0.663742,0.477045,-0.730453,6.273976,-0.492551,0.000396,1.031696,1.004613,-1.045744,-0.831456
2934,Zimbabwe,2003,Developing,-2.596102,4.426698,-0.036497,-0.133960,-0.371370,-2.949358,-0.123970,...,-3.224775,0.232881,-0.603961,6.884478,-0.492623,-0.001958,1.122189,1.115506,-0.993587,-0.742141
2935,Zimbabwe,2002,Developing,-2.564602,-0.738554,-0.044977,-0.042656,-0.371370,-0.316731,-0.184490,...,-0.407639,0.236883,-0.477469,7.494981,-0.520373,-0.206973,-0.823426,-0.791841,-0.950914,-0.593284
2936,Zimbabwe,2001,Developing,-2.512103,4.193377,-0.044977,-0.711394,-0.371370,-0.197067,-0.164869,...,-0.279587,0.088784,-0.308813,7.947934,-0.485949,-0.006346,-0.732932,-0.703127,-0.950914,-0.652827


Remove all nans and save csv

In [21]:
df = df.dropna()
df

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,-0.443615,0.790103,0.268778,-1.133365,-0.335513,-0.635838,-0.110366,...,-3.267459,0.889322,-0.730453,-0.323390,-0.483449,0.343917,2.796324,2.756711,-0.704355,-0.563512
1,Afghanistan,2014,Developing,-0.979112,0.854468,0.285738,-1.133365,-0.334384,-0.755503,-0.168095,...,-1.047897,0.897327,-0.856945,-0.323390,-0.481456,-0.203661,2.864194,2.801068,-0.718580,-0.593284
2,Afghanistan,2013,Developing,-0.979112,0.830331,0.302697,-1.133365,-0.334537,-0.675726,-0.173502,...,-0.877162,0.877314,-0.772617,-0.323390,-0.480121,0.311058,2.909441,2.845424,-0.747029,-0.623055
3,Afghanistan,2012,Developing,-1.021112,0.862513,0.328137,-1.133365,-0.332040,-0.556061,0.032040,...,-0.663742,1.033419,-0.646125,-0.323390,-0.477443,-0.148436,2.954688,2.911960,-0.780219,-0.652827
4,Afghanistan,2011,Developing,-1.052611,0.886650,0.345097,-1.133365,-0.367800,-0.516173,0.051748,...,-0.621059,0.773244,-0.603961,-0.323390,-0.519939,-0.160210,3.022558,2.956317,-0.822893,-0.742141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,Zimbabwe,2004,Developing,-2.617102,4.491063,-0.028017,-0.059930,-0.371370,-0.516173,-0.208296,...,-0.663742,0.477045,-0.730453,6.273976,-0.492551,0.000396,1.031696,1.004613,-1.045744,-0.831456
2934,Zimbabwe,2003,Developing,-2.596102,4.426698,-0.036497,-0.133960,-0.371370,-2.949358,-0.123970,...,-3.224775,0.232881,-0.603961,6.884478,-0.492623,-0.001958,1.122189,1.115506,-0.993587,-0.742141
2935,Zimbabwe,2002,Developing,-2.564602,-0.738554,-0.044977,-0.042656,-0.371370,-0.316731,-0.184490,...,-0.407639,0.236883,-0.477469,7.494981,-0.520373,-0.206973,-0.823426,-0.791841,-0.950914,-0.593284
2936,Zimbabwe,2001,Developing,-2.512103,4.193377,-0.044977,-0.711394,-0.371370,-0.197067,-0.164869,...,-0.279587,0.088784,-0.308813,7.947934,-0.485949,-0.006346,-0.732932,-0.703127,-0.950914,-0.652827


In [22]:
df.to_csv("normalizedData-RemovedNanRows.csv")

This removes about 50% of about data - we need to be more careful about removing these nans...

Fill in Nan values based on this approach: 

If there is other values from the same country, we take the mean of those values and fill in empty data. (if there is none, we use mean) 